# Model diagnostics

This notebook records good practices for effective deep learning model training and diagnostics. Covers the following topics:

1. [First step: Output logs](#first-step-output-logs)
2. [Second step: Log parsing](#second-step-log-parsing)
3. [Third step: Informative plotting](#third-step-informative-plotting)


## First step: Output logs

To understand how your model is doing, you need to collect data on its performance. Model output log files are the place for that. Often repeated advice regarding program logs of any sort can be summarised as "log everything/the more you log the better". But this is a difficult starting point, so here we'll list of the most important parameters and their uses as well as some general notes about good logs.

*todo: logging frequency of different variables?*

### Train/Validation/Test losses

Losses are the most important variables to keep track of because tracking them will tell you how the model is learning. To get the full picture of how your model is doing, it is necessary to track both training and validation/test loss. Comparing training and test loss plotted with regards to epoch can show over-/underfitting phenomena. Plotting losses against learning rate can help choose correct learning rate value or boundaries. 

### Epochs and mini batches

Including the current epoch and mini batches in logs is vital for informative plotting. The plots of different losses and accuracy usually have epochs as the x-axis, to show the "*temporal*" progress of the model makes.

### Time

Logging the time it takes to for example load data or train one batch can help with resource management. Time logs can point out clogs in your model pipeline. For instance GPUs might run idle if the data loading stage takes long. In this case, adding resources (CPU power) to preprocessing stage might help. Furthermore, if some step in the model training takes unreasonably long time, it might hint at a bug in the model code. 

### Accuracy

Accuracy (or similar statistic) is an important measurement of model performance. When an accuracy curve plateaus, it means the model training has converged and it is time to stop training.

### Logging style

When designing log output, always name clearly the values you are printing. Being verbose does not hurt, natural language and descriptive names can on the contrary help you identify variables at the next step. It is also a good idea to write down the units (seconds or milliseconds) and use tab delimiting in your logs to improve readability. Below are few example lines from a log and their Python style formatting strings.


#### Log example
```
Epoch: [1][100/900]	Time 0.486 (1.901)	Data 0.000 (1.394)	Loss total 2.0161 (2.0171)
Epoch: [1][200/900]	Time 7.931 (1.892)	Data 7.389 (1.386)	Loss total 1.9982 (2.0163)
Epoch: [1][300/900]	Time 0.489 (1.882)	Data 0.000 (1.379)	Loss total 2.0468 (2.0176)
Epoch: [1][400/900]	Time 7.007 (1.886)	Data 6.521 (1.384)	Loss total 2.0137 (2.0159)
Epoch: [1][500/900]	Time 0.540 (1.887)	Data 0.000 (1.385)	Loss total 1.9612 (2.0161)
```
#### Formatting

So how do you reproduce the logs above? Below is a minimal working example you can play with. The technical details of how Python formatting works is not the focus here (for that, see [this guide](https://pyformat.info/)). However, here are some general ideas from the code to keep in mind:

1. Create a general class if you need to for example compute statistics of a variable (`AverageMeter`)
2. Split your print to several lines to make your code readable for yourself and others
3. Round your floats to 3-5 significant numbers (`.3f`)
4. Use tab delimiters (`\t`)
5. You can supply objects and access their variables within the formatting string (`batch_time, data_time, loss_meter`, this is supported only in Python3)
6. Set `flush=True` to ensure your prints are outputted right away


In [2]:
# An example of a convenience class for storing a class. You can freely modify this to track other statistics as well.
class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
# Dummy values for the print command below
epoch = 1
i = 100
train_loader = [0] * 900
batch_time = AverageMeter()
data_time = AverageMeter()
loss_meter = AverageMeter()
batch_time.update(0.500)
batch_time.update(0.750)
data_time.update(0.200)
data_time.update(0.333)
loss_meter.update(2.59053)
loss_meter.update(1.49687)

print('Epoch: [{0}][{1}/{2}]\t'
      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
      'Loss total {loss_meter.val:.4f} ({loss_meter.avg:.4f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss_meter=loss_meter), flush=True)


Epoch: [1][100/900]	Time 0.750 (0.625)	Data 0.333 (0.267)	Loss total 1.4969 (2.0437)


## Second step: Log parsing

So now you have lot of logs, but how to efficiently get out the values you want? For example how to find from the previous log example the values after `Loss total`

```
Epoch: [1][100/900]	Time 0.486 (1.901)	Data 0.000 (1.394)	Loss total 2.0161 (2.0171)
Epoch: [1][200/900]	Time 7.931 (1.892)	Data 7.389 (1.386)	Loss total 1.9982 (2.0163)
Epoch: [1][300/900]	Time 0.489 (1.882)	Data 0.000 (1.379)	Loss total 2.0468 (2.0176)
Epoch: [1][400/900]	Time 7.007 (1.886)	Data 6.521 (1.384)	Loss total 2.0137 (2.0159)
Epoch: [1][500/900]	Time 0.540 (1.887)	Data 0.000 (1.385)	Loss total 1.9612 (2.0161)
```
and copying them into a separate file?
```
2.0161 (2.0171)
1.9982 (2.0163)
2.0468 (2.0176)
2.0137 (2.0159)
1.9612 (2.0161)

```

Fortunately there are several powerful command line tools that can help you parse your logs quickly after you understand their basic usage. The following subsections will cover regular expressions, `grep` and `sed`. First is a general programming tool for finding patterns in your text, latter two are command line utilities you can use to find and extract values from your logs. There is also a third command line tool, `awk`, which is a full programming language designed for text processing, but it will not be covered here.  

### Regexp

So you want to find something in a text file? If it is some literal word like *loss*, easy peasy. But what if it is something more complex? You are not looking for a particular word, but for all the numbers or special character in a text. Or all the numbers after a certain word. Essentially, you want to find patterns in the text. Regular expressions are a pattern matching tool designed to do just that. In practice, regular expressions are strings of special characters where each special character is a shorthand for some simple pattern. Combine them correctly and you can match complex patterns like social security numbers, phone numbers from different countries or time and date formats. You could even use regexp to ensure a password fulfills the requirements for a strong password! [More examples here](https://medium.com/factory-mind/regex-cookbook-most-wanted-regex-aa721558c3c1).

To continue our running example, what is the regexp that will match to the words *Loss total* and the two floats after it? The answer is

```
/Loss total [0-9]*\.[0-9]* \([0-9]*\.[0-9]*\)/g
```
Let's break the expression into its constituents.

1. The forward slashes `/.../` indicate the beginning and end of the regexp.
2. The **g**lobal flag `g` at the end of the expression makes the search *global* that is the expression will match all instances of pattern not just that first it can find.
3. `Loss total ` finds exact matches of *Loss total*.
4. Square brackets `[...]` define a range. Here `[0-9]` means numbers ranging from 0 to 9.
5. Asterisk `*` is a quantifier matching to 0 or more instances of the previous token. Here `[0-9]*` reads as *find 0 or more digits in the range from 0 to 9*.
6. `\.` matches to a literal dot character. Adding `\` is necessary because dot alone refers to any character (except line breaks).
7. `\(` and `\)` match to the literal parentheses characters because plain `(...)` has another function in regexps.

To summarise, the example regexp can be read aloud as *match any instance of **Loss total** followed by a float which is followed by another float in parentheses*.

*todo: add an image where the regexp is highlighted according to the above sentence*

If you want to learn more about how to form regexp, [check this tutorial](https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285). Also, designing regexps can be hard, but luckily the internet has [interactive and graphical regexp editors](https://regexr.com/) you can use to check your regexps work. 

### Grep

Now that we have a pattern, we need a tool where we can apply it. Say hello to `grep`. The short definition of `grep`, according to the documentation is
> print lines matching a pattern

A pattern can be just text if you want to find particular words in a log. But you can also supply regexps to `grep` with the `-E` option. The capital E stands for extend regular expressions which have some additional features when compared to the normal regular expressions (flag `-e`). So running the regexp `grep -E 'Loss total [0-9]*\.[0-9]* \([0-9]*\.[0-9]*\)' log_file.txt` on a log file containing the five example lines

```
Epoch: [1][100/900]	Time 0.486 (1.901)	Data 0.000 (1.394)	Loss total 2.0161 (2.0171)
Epoch: [1][200/900]	Time 7.931 (1.892)	Data 7.389 (1.386)	Loss total 1.9982 (2.0163)
Epoch: [1][300/900]	Time 0.489 (1.882)	Data 0.000 (1.379)	Loss total 2.0468 (2.0176)
Epoch: [1][400/900]	Time 7.007 (1.886)	Data 6.521 (1.384)	Loss total 2.0137 (2.0159)
Epoch: [1][500/900]	Time 0.540 (1.887)	Data 0.000 (1.385)	Loss total 1.9612 (2.0161)
```
will produce output where all the matches are highlighted. Note that in `grep` commands the forward slashes `/.../` indicating the beginning and end of a regexp are replaced with apostrophes `'...'`. Flags like `g` are also left out or replaced with options instead. Highlighting is nice, but what if we want to see only the matches and not the whole file? Adding the option `-o` to the command (`grep -oE 'Loss total [0-9]*\.[0-9]* \([0-9]*\.[0-9]*\)' log_file.txt`) will produce

```
Loss total 2.0161 (2.0171)
Loss total 1.9982 (2.0163)
Loss total 2.0468 (2.0176)
Loss total 2.0137 (2.0159)
Loss total 1.9612 (2.0161)
```

Nice! But if we want to copy the values here to plot them, we first have to remove each repetition of `Loss total` and the float in parentheses. While `grep` can print the matches (which is useful in many occasions), it is not exactly suited to print partial matches or to substitute parts of the output with other values. Furthermore, doing the removal manually would just be pain in the ass. Fortunately, there is another tool that can do just what we want. 

### Sed

Meet `sed`. It is a flexible tool that can edit input streams/files in many ways (`sed` is short for **s**tream __ed__itor). The most important and popular editing tool in `sed` is the substitute command which takes the form `sed 's/one/ONE/'`. The four parts of the command are:

```
s          Substitute command
/../../    Delimiter
one        Regular Expression Search Pattern
ONE        Replacement string
```

In short, the substitute command replaces any string matching to the regexp search pattern `one` with the replacement string `ONE`. Great! With small edits to regexp in our example, we can use the substitute command to do what the `grep` command could not. First change is to mark the portion we want to keep (the first float after `Loss total`). This can be done with escaped parentheses `\(...\)` signifying a capturing group. Confusingly enough, with `sed` escaped parentheses mark a capturing group while a lone parenthesis is interpreted as the character itself. Normally with regexps it goes the other way around. Parentheses `(...)` stand for a capturing group and you need to add an escape before literal parentheses in your pattern. The result of these changes is
 
`/Loss total [0-9]*\.[0-9]* \([0-9]*\.[0-9]*\)/` $\rightarrow$ `/Loss total \([0-9]*\.[0-9]*\) ([0-9]*\.[0-9]*)/`

The problem with the current regexp is that `sed` will output the whole line where the match is found and replace only the matching part. So rest of the line will be included in the output unchanged. To avoid this, the second change is to add `.*` in the beginning and the end of the regexp to account for any characters preceding and following our match on the same line (`.*` reads *0 or more instances of any character*). This way the whole line is included in the match, because `sed` operates on a line-by-line basis. After the change we have

`/Loss total \([0-9]*\.[0-9]*\) ([0-9]*\.[0-9]*)/` $\rightarrow$ `/.*Loss total \([0-9]*\.[0-9]*\) ([0-9]*\.[0-9]*).*/`

Now our regexp is ready for `sed`, but what to put in the replacement string? Easy! We will just have to put `\1` there. `\1`refers to the first capturing group i.e. the part that is inside `\(...\)`. If we had more capturing groups `\(...\)`, `\2` would refer to the second capture group starting from left, `\3` to the third etc. Since we only want the float we captured, nothing else is needed. Final form of `sed` command is thus

`sed -n 's/.*Loss total \([0-9]*\.[0-9]*\) ([0-9]*\.[0-9]*).*/\1/p' log_file.txt`

The final touches to the command are the input file `log_file.txt`, the option `-n` and the command `p` at the end of the substitute command. **Without** the latter two, `sed` will output all the lines that do not match to the pattern *as is* in addition to the output from the substitute command. The option `-n` alone will suppress any output from `sed` (=no output). The print command `p` prints the result of the substitute command. Adding it alone will mean that the output of the substitute command is printed twice. Combine the two and `sed` will output only the result of the substitute command. Success! 

```
2.0161
1.9982
2.0468
2.0137
1.9612
```

Similar to the regexp editor above, there is also an [online editor](https://sed.js.org/) for testing your `sed` commands. [Here](http://www.grymoire.com/Unix/Sed.html) is also a longer guide to `sed` commands for those needing it.




## Third step: Informative plotting

### Learning rate search

Learning rate is one of the most important hyperparameters in deep learning models. Just trying out different values might work, but here a good plot can be immensely helpful. Ideally, we would like to have plot like the one below:

<img src="lr_finder.png">

Here the model has been trained for few epochs with a learning rate that changes logarithmically from a given minimum value to a given maximum value. The learning rates are then plotted against a loss, yielding the figure above. From the curve we can easily see, that the values near the minimum were too low because the loss curve stay flat. Then around the learning rate of 4e-4 the loss curve begins to drop dramatically, indicating that the model is learning something. After 1e-3 the curve starts oscillating wildly, telling us that that the learning rate is too big for the model to converge. Based on this, we can pick the initial learning rate from the good range, or use the good range itself in methods like CLR (cyclical learning rate).



### Under- and overfitting